New de-spaggetitification

# SETUP


In [9]:
import sys
import json
from collections import Counter
import time
import re
import copy
from copy import deepcopy

# GLOBAL VARS

In [10]:
replay = "100_sampled_replays_edison/58554463.json"
replay = "solo_mode_replays/to_def_to_atk.json"
replay = "solo_mode_replays/simple_moves_2.json"
replay = "solo_mode_replays/simple_moves_3.json"
replay = "solo_mode_replays/infinite_sirocco.json"
replay = "solo_mode_replays/field_spell2.json"
replay = "solo_mode_replays/field_spell3.json"
replay = "solo_mode_replays/full_field_movement.json"
replay = "solo_mode_replays/move2.json"
replay = "solo_mode_replays/move3.json"

# HELPER FUNCTIONS

In [11]:
# Core functions
def create_object(owner, card_ids, card_obj_ids, card_name, card_state=None, obj_id=None, battle_position=None, face_up=None):
    first_id = card_ids.pop(0)
    card_obj_ids.add(obj_id)
    return {"owner": owner, "name": card_name, "id": first_id, "state": card_state, "object_id": obj_id, "battle_position": battle_position, "face_up": face_up}

def get_log(play):
    return play["log"].get("private_log") or play["log"].get("public_log")

# Utility functions
def json_diff(before, after, path=""):
    """Recursively compare two JSON-like structures and return their differences."""
    diffs = {}

    # Keys that exist in either dict
    all_keys = set(before.keys()) | set(after.keys())
    for key in all_keys:
        current_path = f"{path}.{key}" if path else key

        if key not in before:
            diffs[current_path] = {"added": after[key]}
        elif key not in after:
            diffs[current_path] = {"removed": before[key]}
        else:
            val_before, val_after = before[key], after[key]

            if isinstance(val_before, dict) and isinstance(val_after, dict):
                nested = json_diff(val_before, val_after, current_path)
                diffs.update(nested)
            elif isinstance(val_before, list) and isinstance(val_after, list):
                if val_before != val_after:
                    diffs[current_path] = {
                        "before": val_before,
                        "after": val_after,
                    }
            elif val_before != val_after:
                diffs[current_path] = {
                    "before": val_before,
                    "after": val_after,
                }

    return diffs

# GENERAL ANALYSIS

In [12]:
def parse(json_log):
    persistent_state = {
        # general
        "turn": 0,
        ### me
        "me_life": 8000,
        "me_hand": [],
        "me_GY": [],
        "me_banished": [],
        "me_field": {
            "M-1": None,
            "M-2": None,
            "M-3": None,
            "M-4": None,
            "M-5": None,
            "S-1": None,
            "S-2": None,
            "S-3": None,
            "S-4": None,
            "S-5": None,
            "FSZ": None,
        },
        "me_phase": None,
        ### opponent
        "op_life": 8000,
        "op_hand": [],
        "op_GY": [],
        "op_banished": [],
        "op_field": {
            "M-1": None,
            "M-2": None,
            "M-3": None,
            "M-4": None,
            "M-5": None,
            "S-1": None,
            "S-2": None,
            "S-3": None,
            "S-4": None,
            "S-5": None,
            "FSZ": None,
        },
        "op_phase": None,
    }

    card_ids_me = list(range(1, 100))
    card_ids_op = list(range(100, 200))

    card_obj_ids_me = set()
    card_obj_ids_op = set()

    p1 = json_log["player1"]["username"]
    p2 = json_log["player2"]["username"]

    print(f"Player 1: {p1}, Player 2: {p2}")

    for i, play in enumerate(json_log["plays"]):
        play_type = play.get("play", None)
        if ((play_type == "Duel message" or play_type == "Shuffle hand" 
                or play_type == "Good" or play_type == "Stop viewing") 
                or play_type == "Shuffle deck" or play_type == "Declare"\
                or play_type == "View ED" or play_type == "View GY"\
                or play_type == "View deck" or play_type == "Attack directly"\
                or play_type == "Edit stats"):
            continue
        
        before_state = deepcopy(persistent_state)
        print(f"Doing play {i}: {play}")

        if play_type == "Pick first":
            cards = [(card["name"], card["object_id"]) for card in play["cards"]]
            username = play["username"]
            cards_p1 = cards[:5]
            cards_p2 = cards[5:10]
            if username == p1:
                persistent_state["me_hand"] = [create_object(p1, card_ids_me, card_obj_ids_me, card, "private", obj_id, battle_position=None, face_up=None) for card, obj_id in cards_p1]
                persistent_state["op_hand"] = [create_object(p2, card_ids_op, card_obj_ids_op, card, "private", obj_id, battle_position=None, face_up=None) for card, obj_id in cards_p2]
            else:
                persistent_state["me_hand"] = [create_object(p1, card_ids_me, card_obj_ids_me, card, "private", obj_id, battle_position=None, face_up=None) for card, obj_id in cards_p1]
                persistent_state["op_hand"] = [create_object(p2, card_ids_op, card_obj_ids_op, card, "private", obj_id, battle_position=None, face_up=None) for card, obj_id in cards_p2]

        after_state = deepcopy(persistent_state)
        diff = json_diff(before_state, after_state)

        if diff:
            print(f"\n🔹 Changes after play {i} ({play['play']}):")
            print(json.dumps(diff, indent=2))
        else:
            print(f"\n(no state change after play {i})")

        print(card_obj_ids_me)
        print(card_obj_ids_op)


parse(json.load(open(replay)))


Player 1: illkidnapurmum, Player 2: 
Doing play 0: {'play': 'Pick first', 'seconds': 0, 'cards': [{'def': '1800', 'monster_color': 'Effect', 'arrows': '', 'is_effect': 1, 'scale': 0, 'pic': '1', 'type': 'Beast-Warrior', 'ocg': 1, 'atk': '1200', 'tcg': 1, 'id': 284, 'attribute': 'LIGHT', 'ability': '', 'pendulum': 0, 'flip': 0, 'level': 3, 'custom': 0, 'serial_number': '07183277', 'card_type': 'Monster', 'tcg_limit': 3, 'ocg_limit': 3, 'rush': 0, 'object_id': 1, 'effect': 'Neither player can target "Lightsworn" monsters (anywhere) with card effects. During each of your End Phases: Send the top 2 cards of your Deck to the Graveyard.', 'name': 'Aurkus, Lightsworn Druid', 'pendulum_effect': '', 'treated_as': 'Aurkus, Lightsworn Druid'}, {'def': '200', 'monster_color': 'Effect', 'arrows': '', 'is_effect': 1, 'scale': 0, 'pic': '1', 'type': 'Fairy', 'ocg': 1, 'atk': '2300', 'tcg': 1, 'id': 636, 'attribute': 'LIGHT', 'ability': '', 'pendulum': 0, 'flip': 0, 'level': 5, 'custom': 0, 'serial_nu